In [1]:
import tensorflow as tf
import os
import json
import functools
import csv

2024-10-08 22:28:14.179047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-08 22:28:14.179107: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def _parse(proto, meta):
  """Parses a trajectory from tf.Example."""
  feature_lists = {k: tf.io.VarLenFeature(tf.string)
                   for k in meta['field_names']}
  features = tf.io.parse_single_example(proto, feature_lists)
  out = {}
  for key, field in meta['features'].items():
    data = tf.io.decode_raw(features[key].values, getattr(tf, field['dtype']))
    data = tf.reshape(data, field['shape'])
    if field['type'] == 'static':
      data = tf.tile(data, [meta['trajectory_length'], 1, 1])
    elif field['type'] == 'dynamic_varlen':
      length = tf.io.decode_raw(features['length_'+key].values, tf.int32)
      length = tf.reshape(length, [-1])
      data = tf.RaggedTensor.from_row_lengths(data, row_lengths=length)
    elif field['type'] != 'dynamic':
      raise ValueError('invalid data format')
    out[key] = data
  return out

def load_dataset(path, split):
  """Load dataset."""
  with open(os.path.join(path, 'meta.json'), 'r') as fp:
    meta = json.loads(fp.read())
  ds = tf.data.TFRecordDataset(os.path.join(path, split+'.tfrecord'))
  ds = ds.map(functools.partial(_parse, meta=meta), num_parallel_calls=8)
  ds = ds.prefetch(1)
  return ds

In [9]:
ds = load_dataset("./DATA/cylinder_flow", 'test')

In [4]:
print(type(ds))
print(ds.element_spec)
element = next(iter(ds.take(1).as_numpy_iterator()))
print(element.keys())

csv_file = 'test.csv'

for i in element.keys():
    print(len(element[i]), type(element[i]))


<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
{'cells': TensorSpec(shape=(600, None, 3), dtype=tf.int32, name=None), 'mesh_pos': TensorSpec(shape=(600, None, 2), dtype=tf.float32, name=None), 'node_type': TensorSpec(shape=(600, None, 1), dtype=tf.int32, name=None), 'velocity': TensorSpec(shape=(600, None, 2), dtype=tf.float32, name=None), 'pressure': TensorSpec(shape=(600, None, 1), dtype=tf.float32, name=None)}
dict_keys(['cells', 'mesh_pos', 'node_type', 'velocity', 'pressure'])
600 <class 'numpy.ndarray'>
600 <class 'numpy.ndarray'>
600 <class 'numpy.ndarray'>
600 <class 'numpy.ndarray'>
600 <class 'numpy.ndarray'>


In [10]:
print(type(ds))
element = next(iter(ds))

print(element.keys())
print(type(element))
for name in element.keys():
    
    print(element[name].shape)

print(element["pressure"][0, 1, 0])

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
dict_keys(['cells', 'mesh_pos', 'node_type', 'velocity', 'pressure'])
<class 'dict'>
(600, 3612, 3)
(600, 1923, 2)
(600, 1923, 1)
(600, 1923, 2)
(600, 1923, 1)
tf.Tensor(3.1386888, shape=(), dtype=float32)


In [7]:
num_elements = 1876
num_iter = 600

csv_name = './DATA/cylinder_flow/csv/data_'

header = ["mesh_pos_x", "mesh_pos_y", "velocity_x", "velocity_y", "pressure"]

for i in range(num_iter):
    if i % 10 != 0:
        continue
    file = csv_name + f'{i:03}' + '.csv'
    rows = []
    for j in range(num_elements):
        row = []
        row.append(float(element["mesh_pos"][i, j, 0]))
        row.append(float(element["mesh_pos"][i, j, 1]))
        row.append(float(element["velocity"][i, j, 0]))
        row.append(float(element["velocity"][i, j, 1]))
        row.append(float(element["pressure"][i, j, 0]))
        rows.append(row)
    
    with open(file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header row
        writer.writerows(rows)   # Write all the data rows
    print(i)
        
        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
